In [6]:
import torch.nn.functional as F
import torch
from main_merck_all_real_mixup import get_dataset
from utils import set_seed, get_optimizer, InfIterator
from arguments import get_arguments
from main_origin import get_model
from setenc import get_mixer

import os
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
    

In [7]:
def calc_loss(y_hat, y, test=False):
    return F.mse_loss(y.cuda().squeeze(), y_hat.cuda().squeeze())

def test(args, dataloader, contextloader=None, model=None, mixer_phi=None, embed_type=None, n_t=10, n_c=5):
    model.eval()
    mixer_phi.eval()
    embedding_list = []
    label_list= []
    loss_list = []
    # print('model ', model)
    # print('mixer_phi ', mixer_phi)
    if embed_type == "train_none":
        with torch.no_grad():
            losses = []
            counts = 0
            for i, (x, y) in enumerate(dataloader):
                if i == n_t:
                    break
                
                if i == 0:
                    torch.save(x, 't_x.pt')
                    torch.save(y, 't_y.pt')
                #     return
                

                y_hat, embedding_list, label_list = model(x=x.to(args.device), context=None, mixer_phi=mixer_phi, embedding_list=embedding_list, label_list=label_list, embed_type=embed_type, embed_test=args.embed_test)

                y = y.cuda().squeeze()
                y_hat = y_hat.cuda().squeeze()

                # y_hat = y_hat[:, 0]
                # print(f"in test: {y.size()=} {y_hat.size()=}")

                loss = calc_loss(y_hat, y, test=True)
                loss_scalar = loss.detach().item()
                loss_list.append(torch.full((x.shape[0],), loss_scalar))

                # print('loss_scalar ', loss_scalar)
                # print('x ', x.size(0))
                # print('y_hat ', y_hat)
                # print('embedding_list ', embedding_list)
                losses.append(loss_scalar * x.size(0))
                counts += x.size(0)
                # if i == 0:
                #     torch.save(x, 'tn_x.pt')
                #     torch.save(y, 'tn_y.pt')
        # self.model.eval()
    elif embed_type == "ood1_none" or embed_type == "ood2_none":
        with torch.no_grad():
            losses = []
            counts = 0
            for i, (x, y) in enumerate(dataloader):
                if i == n_t:
                    break
                
                # if i == 0:
                #     torch.save(x, 't_x.pt')
                #     torch.save(y, 't_y.pt')
                #     return
                

                y_hat, embedding_list, label_list = model(x=x.to(args.device), context=None, mixer_phi=mixer_phi, embedding_list=embedding_list, label_list=label_list, embed_type=embed_type, embed_test=args.embed_test)

                y = y.cuda().squeeze()
                y_hat = y_hat.cuda().squeeze()

                # y_hat = y_hat[:, 0]
                # print(f"in test: {y.size()=} {y_hat.size()=}")

                loss = calc_loss(y_hat, y, test=True)
                loss_scalar = loss.detach().item()
                loss_list.append(torch.full((x.shape[0],), loss_scalar))

                # print('loss_scalar ', loss_scalar)
                # print('x ', x.size(0))
                # print('y_hat ', y_hat)
                # print('embedding_list ', embedding_list)
                losses.append(loss_scalar * x.size(0))
                counts += x.size(0)
                # if i == 0:
                #     torch.save(x, 'tn_x.pt')
                #     torch.save(y, 'tn_y.pt')
        # self.model.eval()
    elif embed_type == "train_context":
        with torch.no_grad():
            losses = []
            counts = 0
            for i, (x, y) in enumerate(dataloader):
                if i == n_t:
                    break
                
                context_samples = []
                
                if i == 0:
                    torch.save(x, 'tc_x.pt')
                    torch.save(y, 'tc_y.pt')

                for i_c, (x_c, y_c) in enumerate(contextloader):
                    if i_c == n_c:
                        break
                    x_c = x_c.reshape(args.batch_size, -1, x_c.size(-1))
                    if args.n_context > 1:
                        n = torch.randint(1, x_c.size(1), size=(1,)).item()
                        x_c = x_c[:, :n]
                        
                    if i == 0 and i_c == 0:
                        torch.save(x_c, 'tc_x_c.pt')
                        torch.save(y_c, 'tc_y_c.pt')
                        
                        
                    # print('$$$$$$')
                    # print('x_c ', x_c.shape)
                    # print('x ', x.shape)
                    
                    # return
                    # context_samples.append(x_c)
                
                # context_samples = torch.cat(context_samples, dim=0).to(args.device)
                # context_samples = context_samples.reshape(args.batch_size, -1, x_c.size(-1))
                
                # print('### context_samples ', context_samples)
                # torch.save(context_samples, f'context_samples_{i}.pth')
                    y_hat, embedding_list, label_list = model(x=x.to(args.device), context=x_c.to(args.device), mixer_phi=mixer_phi, embedding_list=embedding_list, label_list=label_list, embed_type=embed_type, embed_test=args.embed_test)

                    y = y.cuda().squeeze()
                    y_hat = y_hat.cuda().squeeze()

                    # y_hat = y_hat[:, 0]
                    # print(f"in test: {y.size()=} {y_hat.size()=}")

                    loss = calc_loss(y_hat, y, test=True)
                    loss_list.append(torch.full((x.shape[0],), loss.detach().item()))

                    losses.append(loss.item() * x.size(0))
                    counts += x.size(0)
    elif embed_type == "context_none":
        with torch.no_grad():
            losses = []
            counts = 0
            for i, (x, y) in enumerate(dataloader):
                if i == n_t:
                    break
                
                context_samples = []
                
                if i == 0:
                    torch.save(x, 'c_x.pt')
                    torch.save(y, 'c_y.pt')

                for i_c, (x_c, y_c) in enumerate(contextloader):
                    if i_c == n_c:
                        break
                    x_c = x_c.reshape(args.batch_size, -1, x_c.size(-1))
                    if args.n_context > 1:
                        n = torch.randint(1, x_c.size(1), size=(1,)).item()
                        x_c = x_c[:, :n]
                    
                    if i == 0 and i_c == 0:
                        torch.save(x_c, 'c_x_c.pt')
                        torch.save(y_c, 'c_y_c.pt')
                    # print('====')
                    # print('x_c ', x_c.shape)
                    # print('x ', x.shape)
                    
                    # return
                    
                    # context_samples.append(x_c)
                    
                    # B, S, H = x_c.size() <- did this in model
                    # x_c = x_c.view(B*S, H)
                
                # context_samples = torch.cat(context_samples, dim=0).to(args.device)
                # context_samples = context_samples.reshape(args.batch_size, -1, x_c.size(-1))
                
                # print('### context_samples ', context_samples)
                # torch.save(context_samples, f'context_samples_{i}.pth')
                    y_hat, embedding_list, label_list = model(x=x_c.to(args.device), context=x_c.to(args.device), mixer_phi=mixer_phi, embedding_list=embedding_list, label_list=label_list, embed_type=embed_type, embed_test=args.embed_test)

                    y = y.cuda().squeeze()
                    y_hat = y_hat.cuda().squeeze()

                    # y_hat = y_hat[:, 0]
                    # print(f"in test: {y.size()=} {y_hat.size()=}")

                    loss = calc_loss(y_hat, y, test=True)
                    loss_list.append(torch.full((x.shape[0],), loss.detach().item()))

                    losses.append(loss.item() * x.size(0))
                    counts += x.size(0)
    mse = sum(losses) / counts
    return mse, embedding_list, label_list, loss_list

In [8]:

def save_one_datapoint_features(args, model, mixer_phi, embed_type, n_t, n_c):
    assert args.model_no_context == False
    
    args.embed_test = 'lastlayer_ours_best'
    
    path = f"/c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNct{args.model_no_context}_RYV1_MIXUP/{args.embed_test}_"
    os.makedirs(path, exist_ok=True)
    
    
    
    f_path = f'/c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNct{args.model_no_context}_RYV1_MIXUP/{args.embed_test}_/{args.sencoder}_{args.dataset}_{args.vec_type}_{n_t}_{n_c}_{embed_type}.npz'
    
    
    if os.path.exists(f_path):
        print(f"⏩ {f_path} already exists. Skipping...") 
        return
    
    set_seed(0)
    args.batch_size = 1
    args.tsne_plot = True # because of get_dataset
    all_candidates = ['hivprot', 'dpp4', 'nk1']
    args.specify_ood_dataset = [d for d in all_candidates if d != args.dataset]
    trainloader_test, _, mvalidloader_test, _, contextloader_test, ood1_trainloader_test, ood2_trainloader_test = get_dataset(args=args, test=True)
    
    if "ood" not in embed_type:
        mse, embedding_list, label_list, loss_list = test(args=args, dataloader=trainloader_test, contextloader=contextloader_test, model=model, mixer_phi=mixer_phi, embed_type=embed_type, n_t=n_t, n_c=n_c)
    elif 'ood1' in embed_type:
        mse, embedding_list, label_list, loss_list = test(args=args, dataloader=ood1_trainloader_test, contextloader=contextloader_test, model=model, mixer_phi=mixer_phi, embed_type=embed_type, n_t=n_t, n_c=n_c)
    elif 'ood2' in embed_type:
        mse, embedding_list, label_list, loss_list = test(args=args, dataloader=ood2_trainloader_test, contextloader=contextloader_test, model=model, mixer_phi=mixer_phi, embed_type=embed_type, n_t=n_t, n_c=n_c)
    else:
        raise Exception()
    
    # for i, emb in enumerate(embedding_list):
    #     print(f"Tensor {i}: {emb.shape}")
    
    all_embeddings = torch.cat(embedding_list, dim=0)
    all_labels = np.concatenate(label_list, axis=0)
    all_losses = torch.cat(loss_list, dim=0)

    all_labels = torch.tensor(all_labels)
    all_losses = torch.tensor(all_losses)
                
    all_embeddings_np = all_embeddings.numpy()
    all_labels_np = all_labels.numpy()
    all_losses_np = all_losses.numpy()
    
    
    np.savez(f_path, embeddings=all_embeddings_np, labels=all_labels_np, losses=all_losses_np)
    print(f'>>> saved {f_path}')
    
    if 'ood1' in embed_type:
        ood1_trainloader_test._iterator._shutdown_workers()
    elif 'ood2' in embed_type:
        ood2_trainloader_test._iterator._shutdown_workers()
    else:
        trainloader_test._iterator._shutdown_workers()
        if 'context' in embed_type:
            contextloader_test._iterator._shutdown_workers()


In [9]:
def save_features(data):

    model = data['model']
    mixer_phi = data['mixer_phi']
    optimizer = data['optimizer']
    # mixer_optimizer = data['mixer_optimizer']
    
    ltmse, lvmse, vmse, tmse = data['ltmse'], data['lvmse'], data['vmse'], data['tmse ']
    
    print('>> ltmse ', ltmse)
    print('>> tmse ', tmse)
    print('>> lvmse ', lvmse)
    print('>> vmse ', vmse)
    
    args_ = data['args']

    args = get_arguments()

    for k, v in args_.items():
        setattr(args, k, v)
        
    os.environ['MIX_TYPE'] = 'MIXUP'

    set_seed(0)

    model = get_model(args=args)
    mixer_phi = get_mixer(args=args)
    # optimizer = get_optimizer(optimizer=args.optimizer, model=model, lr=args.lr, wd=args.wd)
    # optimizermixer = None if mixer_phi is None else get_optimizer(optimizer=args.optimizer, model=mixer_phi, lr=args.clr, wd=args.cwd)

    model.load_state_dict(data['model'])
    mixer_phi.load_state_dict(data['mixer_phi'])
    # optimizer.load_state_dict(data['optimizer'])
    # mixer_optimizer.load_state_dict(data['mixer_optimizer'])

    model = model.to(args.device)
    mixer_phi = mixer_phi.to(args.device)
    
    # for (n_t, n_c) in [(5, 10), (5, 100), (10, 5), (10, 100), (100, 5), (100, 10)]:
    # for (n_t, n_c) in [(5, 10), (5, 50), (5, 100), (10, 5), (10, 40), (10, 100), (40, 5), (40, 10), (100, 5),]:
    for (n_t, n_c) in [(10, 40), (10, 100)]:
        save_one_datapoint_features(args, model, mixer_phi, "train_none", n_t, n_c)
        save_one_datapoint_features(args, model, mixer_phi, "context_none", n_t, n_c)
        save_one_datapoint_features(args, model, mixer_phi, "train_context", n_t, n_c)
    save_one_datapoint_features(args, model, mixer_phi, "ood1_none", 500, 10)
    save_one_datapoint_features(args, model, mixer_phi, "ood2_none", 500, 10)

In [10]:
import os
import torch

# Directory containing .pth files
tsne_model_dir = '/c2/jinakim/Drug_Discovery_j/tsne_model2_mNctFalse_RYV1_mixTrue_MIXUP/ours_best/'
os.makedirs(tsne_model_dir, exist_ok=True)
# List all .pth files
pth_files = sorted([f for f in os.listdir(tsne_model_dir) if f.endswith('.pth')])

# Load each file
for i, f in enumerate(pth_files):
    # if i > 5:
    #     break
    file_path = os.path.join(tsne_model_dir, f)
    print(f"🚀 Loading {file_path}")
    
    data = torch.load(file_path)
    save_features(data)
    # Now 'data' contains the loaded model or state dict or whatever was saved
    # You can process it here if needed
    # For example, just printing some keys if it's a checkpoint
    # if isinstance(data, dict):
    #     print(f"✅ Loaded {f}: keys = {list(data.keys())}")
    # else:
    #     print(f"✅ Loaded {f}: type = {type(data)}")

print("\n🏁 All models loaded.")




🚀 Loading /c2/jinakim/Drug_Discovery_j/tsne_model2_mNctFalse_RYV1_mixTrue_MIXUP/ours_best/Model_strans_nk1_bit_['None'].pth
>> ltmse  0.4454459672567548
>> tmse  0.4454459672567548
>> lvmse  0.30273041625817615
>> vmse  0.30273041625817615
loading set transformer layer='max'
STEncoder(
  (encoder): Sequential(
    (0): SAB(
      (mab): MAB(
        (fc_q): Linear(in_features=512, out_features=512, bias=True)
        (fc_k): Linear(in_features=512, out_features=512, bias=True)
        (fc_v): Linear(in_features=512, out_features=512, bias=True)
        (fc_o): Linear(in_features=512, out_features=512, bias=True)
      )
    )
  )
  (proj): Linear(in_features=512, out_features=512, bias=True)
)
Inner args.dataset='nk1' args.vec_type='bit'


/tmp/ipykernel_586647/140253672.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_losses = torch.tensor(all_losses)


>>> saved /c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1_MIXUP/lastlayer_ours_best_/strans_nk1_bit_10_40_train_none.npz
Inner args.dataset='nk1' args.vec_type='bit'
>>> saved /c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1_MIXUP/lastlayer_ours_best_/strans_nk1_bit_10_40_context_none.npz
Inner args.dataset='nk1' args.vec_type='bit'
>>> saved /c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1_MIXUP/lastlayer_ours_best_/strans_nk1_bit_10_40_train_context.npz
Inner args.dataset='nk1' args.vec_type='bit'
>>> saved /c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1_MIXUP/lastlayer_ours_best_/strans_nk1_bit_10_100_train_none.npz
Inner args.dataset='nk1' args.vec_type='bit'
>>> saved /c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1_MIXUP/lastlayer_ours_best_/strans_nk1_bit_10_100_context_none.npz
Inner args.dataset='nk1' args.vec_type='bit'
>>> saved /c2/jinakim/Drug_Discovery_j/analysis/tsne_las

In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# Try to use GPU TSNE (cuML), fallback to CPU TSNE (openTSNE)
# try:
#     import cupy as cp
#     from cuml.manifold import TSNE as cuTSNE
#     gpu_available = True
#     print("✅ Using GPU cuML TSNE")
# except ImportError:
from openTSNE import TSNE as cpuTSNE
gpu_available = False
print("⚠️ cuML not available, falling back to CPU openTSNE")

# Set your directory
tsne_dir = '/c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1_MIXUP/lastlayer_ours_best_/'

# List all .npz files
files = sorted([f for f in os.listdir(tsne_dir) if f.endswith('.npz')])

# --- Group files by their starting prefix (before 3rd underscore) ---
groups = defaultdict(list)
for f in files:
    parts = f.split('_')
    prefix = '_'.join(parts[:5])  # e.g., dsets_dpp4_count
    groups[prefix].append(f)

# --- Process each group ---
for prefix, group_files in groups.items():
    save_path = os.path.join(tsne_dir, f"{prefix}_tsne_combined.pdf")
    
    # if "dpp4_bit" in save_path:
    #     continue
    # --- Skip if already exists ---
    if os.path.exists(save_path):
        print(f"⏩ {save_path} already exists. Skipping...")
        continue

    print(f"\n🚀 Processing group {prefix}")

    all_embeddings = []
    all_labels = []

    for f in group_files:
        file_path = os.path.join(tsne_dir, f)
        
        #####
        if 'ood' in file_path:
            continue
        
        # if 'train' not in file_path and 'context' not in file_path:
        #     continue
        #####
        
        data = np.load(file_path)
        
        embeddings = data['embeddings']
        labels = data['labels']

        print(f"✅ Loaded {f}: embeddings {embeddings.shape}, labels {labels.shape}")

        all_embeddings.append(embeddings)
        all_labels.append(labels)

    if len(all_embeddings) == 0 or len(all_labels) == 0:
        continue
    
    # Concatenate all
    all_embeddings = np.concatenate(all_embeddings, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    print(f"✅ Combined embeddings shape: {all_embeddings.shape}")

    # --- Run t-SNE ---
    # if gpu_available:
    #     embeddings_gpu = cp.asarray(all_embeddings)
    #     tsne = cuTSNE(n_components=2, random_state=42)
    #     embeddings_2d_gpu = tsne.fit_transform(embeddings_gpu)
    #     embeddings_2d = cp.asnumpy(embeddings_2d_gpu)
    # else:
    tsne = cpuTSNE(n_components=2, n_jobs=8, random_state=42)
    embeddings_2d = tsne.fit(all_embeddings)

    # --- Plot ---
    import matplotlib.patches as mpatches
    import matplotlib.pyplot as plt

    # Set global font
    plt.rcParams['font.family'] = 'DejaVu Sans'
    plt.rcParams['font.size'] = 10

    # --- t-SNE Scatter Plot with Nice Legend and Slight Grid ---

    plt.figure(figsize=(8, 6))

    # Define color mapping
    color_map = {0: '#ffb347', 1: '#0000CD', -1: '#DDA0DD'}  # orange and blue
    colors = [color_map[label] for label in all_labels]

    # Set point sizes
    sizes = [40 if label == 1 else 15 for label in all_labels]

    # Scatter plot
    scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=colors, s=sizes, alpha=0.8)

    # Slight soft grid
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.3)

    # Hide axis labels but keep grid
    plt.gca().set_xticklabels([])
    plt.gca().set_yticklabels([])
    plt.xlabel("")
    plt.ylabel("")
    plt.box(False)

    # --- Add better legend ---
    import matplotlib.lines as mlines

    # Define custom legend handles (use Line2D for circles)
    orange_circle = mlines.Line2D([], [], color='#ffb347', marker='o', linestyle='None', markersize=8, label='ours')
    blue_circle = mlines.Line2D([], [], color='#0000CD', marker='o', linestyle='None', markersize=8, label='ours (w/o context)')
    red_circle = mlines.Line2D([], [], color='#DDA0DD', marker='o', linestyle='None', markersize=8, label='ours (context)')

    # Add legend inside plot (upper right)
    plt.legend(handles=[orange_circle, blue_circle, red_circle],
            loc='upper right',  # inside the plot, top right
            framealpha=0.6,
            prop={'size': 12},
            handletextpad=0.4,
            borderpad=0.5)

    # Tight layout
    plt.tight_layout()

    # Save figure
    plt.savefig(save_path, dpi=300)
    print(f"✅ Saved t-SNE scatter plot with nice legend to {save_path}")
    plt.close()


⚠️ cuML not available, falling back to CPU openTSNE

🚀 Processing group strans_nk1_bit_10_100
✅ Loaded strans_nk1_bit_10_100_context_none.npz: embeddings (1000, 32), labels (1000,)
✅ Loaded strans_nk1_bit_10_100_train_context.npz: embeddings (1000, 32), labels (1000,)
✅ Loaded strans_nk1_bit_10_100_train_none.npz: embeddings (10, 32), labels (10,)
✅ Combined embeddings shape: (2010, 32)
✅ Saved t-SNE scatter plot with nice legend to /c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1_MIXUP/lastlayer_ours_best_/strans_nk1_bit_10_100_tsne_combined.pdf

🚀 Processing group strans_nk1_bit_10_40
✅ Loaded strans_nk1_bit_10_40_context_none.npz: embeddings (400, 32), labels (400,)
✅ Loaded strans_nk1_bit_10_40_train_context.npz: embeddings (400, 32), labels (400,)
✅ Loaded strans_nk1_bit_10_40_train_none.npz: embeddings (10, 32), labels (10,)
✅ Combined embeddings shape: (810, 32)
✅ Saved t-SNE scatter plot with nice legend to /c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REA

In [12]:
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from collections import defaultdict

# # Try to use GPU TSNE (cuML), fallback to CPU TSNE (openTSNE)
# # try:
# #     import cupy as cp
# #     from cuml.manifold import TSNE as cuTSNE
# #     gpu_available = True
# #     print("✅ Using GPU cuML TSNE")
# # except ImportError:
# from openTSNE import TSNE as cpuTSNE
# gpu_available = False
# print("⚠️ cuML not available, falling back to CPU openTSNE")

# # Set your directory
# tsne_dir = '/c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1/lastlayer_ours_best_/'

# # List all .npz files
# files = sorted([f for f in os.listdir(tsne_dir) if f.endswith('.npz')])

# # --- Group files by their starting prefix (before 3rd underscore) ---
# groups = defaultdict(list)
# for f in files:
#     parts = f.split('_')
    
#     prefix = '_'.join(parts[:5])  # e.g., dsets_dpp4_count
    
#     if 'ood' not in f:
#         groups[prefix].append(f)
    
#     prefix_ = '_'.join(parts[:3])
#     if prefix_ in prefix and 'ood' in f:
#         groups[prefix].append(f)

# # --- Process each group ---
# for prefix, group_files in groups.items():
#     save_path = os.path.join(tsne_dir, f"{prefix}_tsne_combined_all.pdf")
    
#     # if "dpp4_bit" in save_path:
#     #     continue
#     # --- Skip if already exists ---
#     if os.path.exists(save_path):
#         print(f"⏩ {save_path} already exists. Skipping...")
#         continue

#     print(f"\n🚀 Processing group {prefix}")

#     all_embeddings = []
#     all_labels = []

#     for f in group_files:
#         file_path = os.path.join(tsne_dir, f)
        
#         if len(group_files) != 5:
#             print('group_files ', len(group_files))
#             continue
#         #####
#         # if 'ood' in file_path:
#         #     continue
        
#         # if 'train' not in file_path and 'context' not in file_path:
#         #     continue
#         #####
        
#         data = np.load(file_path)
        
#         embeddings = data['embeddings']
#         labels = data['labels']

#         print(f"✅ Loaded {f}: embeddings {embeddings.shape}, labels {labels.shape}")

#         all_embeddings.append(embeddings)
#         all_labels.append(labels)

#     if len(all_embeddings) == 0 or len(all_labels) == 0:
#         continue
    
#     # Concatenate all
#     all_embeddings = np.concatenate(all_embeddings, axis=0)
#     all_labels = np.concatenate(all_labels, axis=0)

#     print(f"✅ Combined embeddings shape: {all_embeddings.shape}")

#     # --- Run t-SNE ---
#     # if gpu_available:
#     #     embeddings_gpu = cp.asarray(all_embeddings)
#     #     tsne = cuTSNE(n_components=2, random_state=42)
#     #     embeddings_2d_gpu = tsne.fit_transform(embeddings_gpu)
#     #     embeddings_2d = cp.asnumpy(embeddings_2d_gpu)
#     # else:
#     tsne = cpuTSNE(n_components=2, n_jobs=8, random_state=42)
#     embeddings_2d = tsne.fit(all_embeddings)

#     # --- Plot ---
#     import matplotlib.patches as mpatches
#     import matplotlib.pyplot as plt

#     # Set global font
#     plt.rcParams['font.family'] = 'DejaVu Sans'
#     plt.rcParams['font.size'] = 10

#     # --- t-SNE Scatter Plot with Nice Legend and Slight Grid ---

#     plt.figure(figsize=(8, 6))

#     # Define color mapping
#     color_map = {0: '#ffb347', 1: '#0000CD', -1: '#DDA0DD', 3:'#48b33c', 4:'#3cadb3'}  # orange and blue
#     colors = [color_map[label] for label in all_labels]

#     # Set point sizes
#     sizes = [40 if label == 1 else 15 for label in all_labels]

#     # Scatter plot
#     scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=colors, s=sizes, alpha=0.8)

#     # Slight soft grid
#     plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.3)

#     # Hide axis labels but keep grid
#     plt.gca().set_xticklabels([])
#     plt.gca().set_yticklabels([])
#     plt.xlabel("")
#     plt.ylabel("")
#     plt.box(False)

#     # --- Add better legend ---
#     import matplotlib.lines as mlines

#     # Define custom legend handles (use Line2D for circles)
#     orange_circle = mlines.Line2D([], [], color='#ffb347', marker='o', linestyle='None', markersize=8, label='ours')
#     blue_circle = mlines.Line2D([], [], color='#0000CD', marker='o', linestyle='None', markersize=8, label='ours (w/o context)')
#     red_circle = mlines.Line2D([], [], color='#DDA0DD', marker='o', linestyle='None', markersize=8, label='ours (context)')
#     green_circle = mlines.Line2D([], [], color='#48b33c', marker='o', linestyle='None', markersize=8, label='OOD1')
#     bluegreen_circle = mlines.Line2D([], [], color='#3cadb3', marker='o', linestyle='None', markersize=8, label='OOD2')

#     # Add legend inside plot (upper right)
#     plt.legend(handles=[orange_circle, blue_circle, red_circle, green_circle, bluegreen_circle],
#             loc='upper right',  # inside the plot, top right
#             framealpha=0.6,
#             prop={'size': 12},
#             handletextpad=0.4,
#             borderpad=0.5)

#     # Tight layout
#     plt.tight_layout()

#     # Save figure
#     plt.savefig(save_path, dpi=300)
#     print(f"✅ Saved t-SNE scatter plot with nice legend to {save_path}")
#     plt.close()


In [14]:
print('TODO change path')
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
import matplotlib as mpl
from openTSNE import TSNE as cpuTSNE
from matplotlib.ticker import FormatStrFormatter

print("⚠️ cuML not available, falling back to CPU openTSNE")

# --- Global matplotlib settings ---
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 13
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['figure.titlesize'] = 16
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['axes.titleweight'] = 'bold'
mpl.rcParams['axes.labelweight'] = 'bold'
mpl.rcParams['legend.title_fontsize'] = 13
mpl.rcParams['grid.alpha'] = 0.3
mpl.rcParams['grid.linestyle'] = '--'

# --- Set your directory ---
tsne_dir = '/c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1_MIXUP/lastlayer_ours_best_/'

# List all .npz files
files = sorted([f for f in os.listdir(tsne_dir) if f.endswith('.npz')])

# --- Group files manually ---
groups = defaultdict(list)
for f in files:
    if 'strans_nk1_bit_10_100' in f:
        groups['strans_nk1_bit_10_100'].append(f)
    elif 'strans_nk1_bit_500_10_ood1' in f:
        groups['strans_nk1_bit_10_100'].append(f)
    elif 'strans_nk1_bit_500_10_ood2' in f:
        groups['strans_nk1_bit_10_100'].append(f)

# --- Process each group ---
for prefix, group_files in groups.items():
    base_save_path = os.path.join(tsne_dir, f"{prefix}_tsne_combined")

    print(f"\n🚀 Processing group {prefix}")

    all_embeddings = []
    all_labels = []
    all_losses = []

    for f in group_files:
        file_path = os.path.join(tsne_dir, f)

        if len(group_files) != 5:
            print('⚠️ Skipping group_files', len(group_files))
            continue

        data = np.load(file_path)
        embeddings = data['embeddings']
        labels = data['labels']
        losses = data['losses']

        print(f"✅ Loaded {f}: embeddings {embeddings.shape}, labels {labels.shape}, losses {losses.shape}")

        all_embeddings.append(embeddings)
        all_labels.append(labels)
        all_losses.append(losses)

    if len(all_embeddings) == 0 or len(all_labels) == 0:
        continue

    # Concatenate all
    all_embeddings = np.concatenate(all_embeddings, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    all_losses = np.concatenate(all_losses, axis=0)

    print(f"✅ Combined embeddings shape: {all_embeddings.shape}, labels shape: {all_labels.shape}, losses shape: {all_losses.shape}")

    # --- Run t-SNE ---
    tsne = cpuTSNE(n_components=2, n_jobs=8, random_state=42)
    embeddings_2d = tsne.fit(all_embeddings)

    # --- Fixed normal class colors ---
    fixed_color_map = {0: '#ffb347', 1: '#0000CD', -1: '#DDA0DD'}

    # --- Function to plot each OOD separately ---
    def plot_one_ood(ood_mask, ood_label):
        if not np.any(ood_mask):
            print(f"⚠️ No OOD{ood_label} points to plot.")
            return

        losses_ood = all_losses[ood_mask]
        upper_clip = np.percentile(losses_ood, 95)
        norm = mcolors.Normalize(vmin=losses_ood.min(), vmax=upper_clip)
        cmap = cm.get_cmap('viridis')

        plt.figure(figsize=(8, 6))

        # --- Prepare points ---
        colors = []
        alphas = []
        sizes = []

        for label, loss in zip(all_labels, all_losses):
            if label in fixed_color_map:
                colors.append(fixed_color_map[label])
                alphas.append(0.4)  # lighter background
                sizes.append(20 if label in [0, -1] else 40)
            elif label == ood_label:
                clipped_loss = min(loss, upper_clip)
                colors.append(cmap(norm(clipped_loss)))
                alphas.append(1.0)
                sizes.append(30)
            else:
                colors.append(None)
                alphas.append(None)
                sizes.append(None)

        # --- Plot OOD points first for colorbar ---
        ood_coords = []
        ood_colors = []
        for i in range(len(embeddings_2d)):
            if all_labels[i] == ood_label:
                ood_coords.append(embeddings_2d[i])
                clipped_loss = min(all_losses[i], upper_clip)
                ood_colors.append(clipped_loss)

        if ood_coords:
            ood_coords = np.array(ood_coords)
            ood_colors = np.array(ood_colors)

            scatter = plt.scatter(ood_coords[:, 0], ood_coords[:, 1],
                                  c=ood_colors, cmap=cmap, norm=norm, s=30, alpha=1.0)

        # --- Plot normal points ---
        for i in range(len(embeddings_2d)):
            if colors[i] is None or all_labels[i] == ood_label:
                continue
            plt.scatter(embeddings_2d[i, 0], embeddings_2d[i, 1],
                        color=colors[i], s=sizes[i], alpha=alphas[i])

        plt.grid(True)
        plt.gca().set_xticklabels([])
        plt.gca().set_yticklabels([])
        plt.xlabel("")
        plt.ylabel("")
        plt.box(False)

        # --- Legend ---
        orange_circle = mlines.Line2D([], [], color='#ffb347', marker='o', linestyle='None', markersize=8, label='ours')
        blue_circle = mlines.Line2D([], [], color='#0000CD', marker='o', linestyle='None', markersize=8, label='ours (w/o context)')
        red_circle = mlines.Line2D([], [], color='#DDA0DD', marker='o', linestyle='None', markersize=8, label='ours (context)')

        plt.legend(handles=[orange_circle, blue_circle, red_circle],
                   loc='upper right',
                   framealpha=0.6,
                   prop={'size': 11},
                   handletextpad=0.4,
                   borderpad=0.5)

        # --- Prettier colorbar ---
        if ood_coords.size > 0:
            cbar = plt.colorbar(scatter, shrink=0.65, pad=0.01, aspect=35)
            cbar.set_label('Out-Of-Distribution Loss Value', fontsize=13, weight='bold', labelpad=8)
            cbar.outline.set_visible(False)  # no outer box
            cbar.ax.tick_params(labelsize=9, width=0.5, length=3)  # slim ticks
            if losses_ood.max() > 10:
                cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.1e'))

        # --- Title and layout ---
        plt.title(f'TSNE Plot - OOD{ood_label}', fontsize=16, weight='bold', pad=15)
        plt.tight_layout()

        final_save_path = f"{base_save_path}_ood{ood_label}.pdf"
        plt.savefig(final_save_path, dpi=300)
        print(f"✅ Saved clean t-SNE plot for OOD{ood_label} to {final_save_path}")
        plt.close()

        # --- Loss statistics ---
        mean_loss = losses_ood.mean()
        var_loss = losses_ood.var(ddof=0)  # use ddof=0 for population variance
        median_loss = np.median(losses_ood)

        print(f"📈 Loss statistics for OOD{ood_label}:")
        print(f"  Mean   : {mean_loss:.6f}")
        print(f"  Variance: {var_loss:.6f}")
        print(f"  Median : {median_loss:.6f}")

    # --- Plot separately for OOD1 and OOD2 ---
    ood1_mask = (all_labels == 3)
    ood2_mask = (all_labels == 4)

    plot_one_ood(ood1_mask, ood_label=3)
    plot_one_ood(ood2_mask, ood_label=4)


TODO change path
⚠️ cuML not available, falling back to CPU openTSNE

🚀 Processing group strans_nk1_bit_10_100
✅ Loaded strans_nk1_bit_10_100_context_none.npz: embeddings (1000, 32), labels (1000,), losses (1000,)
✅ Loaded strans_nk1_bit_10_100_train_context.npz: embeddings (1000, 32), labels (1000,), losses (1000,)
✅ Loaded strans_nk1_bit_10_100_train_none.npz: embeddings (10, 32), labels (10,), losses (10,)
✅ Loaded strans_nk1_bit_500_10_ood1_none.npz: embeddings (500, 32), labels (500,), losses (500,)
✅ Loaded strans_nk1_bit_500_10_ood2_none.npz: embeddings (500, 32), labels (500,), losses (500,)
✅ Combined embeddings shape: (3010, 32), labels shape: (3010,), losses shape: (3010,)


/tmp/ipykernel_586647/4238003690.py:101: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('viridis')


✅ Saved clean t-SNE plot for OOD3 to /c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1_MIXUP/lastlayer_ours_best_/strans_nk1_bit_10_100_tsne_combined_ood3.pdf
📈 Loss statistics for OOD3:
  Mean   : 1.849176
  Variance: 4.449352
  Median : 1.019940


/tmp/ipykernel_586647/4238003690.py:101: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('viridis')


✅ Saved clean t-SNE plot for OOD4 to /c2/jinakim/Drug_Discovery_j/analysis/tsne_last_REAL2_mNctFalse_RYV1_MIXUP/lastlayer_ours_best_/strans_nk1_bit_10_100_tsne_combined_ood4.pdf
📈 Loss statistics for OOD4:
  Mean   : 1.242078
  Variance: 2.314384
  Median : 0.562863
